In [1]:
import pandas as pd
import numpy as np
import copy
import datetime
import multiprocessing as mp
from matplotlib import pyplot as plt

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [37]:
arquivos = ['matomo_completo_30062020.csv','matomo_completo_24-26062020.csv','matomo_completo_22-23062020.csv','matomo_completo_17-19062020.csv','matomo_completo_15-16062020.csv','matomo_completo_10-12062020.csv','matomo_completo_08-09062020.csv','matomo_completo_04-05062020.csv','matomo_completo_02062020.csv']
arquivos = ['matomo_completo_01-03062020.csv']
#arquivos.append('matomo_completo_01-03062020.csv')
#arquivos.append('matomo_completo_04-05062020.csv')
##arquivos.append('matomo_completo_08-09062020.csv')
#arquivos.append('matomo_completo_10-12062020.csv')
#arquivos.append('matomo_completo_15-16062020.csv')
#arquivos.append('matomo_completo_17-19062020.csv')
#arquivos.append('matomo_completo_22-23062020.csv')
#arquivos.append('matomo_completo_24-26062020.csv')
#arquivos.append('matomo_completo_29062020.csv')


In [4]:
arquivos

['matomo_completo_30062020.csv']

In [38]:
df = pd.DataFrame([],columns=['CD_VIS', 'DT_ACAO', 'HR_LCL_VIST', 'TX_IP_VIS', 'CD_UOR', 'NM_UOR_RDZ',
       'HR_ACAO', 'TEMPO_RESPOSTA', 'TX_IDFC_USU', 'UF', 'NM_TIP_ACAO',
       'HORA_TRS', 'DIA_MES', 'DIA_PICO', 'DIA_UTIL', 'CONTEXTO', 'APP',
       'HORA_LOCAL', 'DT_ACAO_DATETIME', 'HORA_COMERCIAL',
       'TEMPO_RESPOSTA_SEGUNDOS', 'TIMEOUT'])
for arq in arquivos:
    df_temp = pd.read_csv('../data/'+arq,sep=';',low_memory=False)#.sample(2000,random_state=1024)
    
    df = pd.concat([df,df_temp],sort=False)

df_temp =[]
    

In [39]:
len(df)

14795558

In [7]:
df.columns

Index(['CD_VIS', 'DT_ACAO', 'HR_LCL_VIST', 'TX_IP_VIS', 'CD_UOR', 'NM_UOR_RDZ',
       'HR_ACAO', 'TEMPO_RESPOSTA', 'TX_IDFC_USU', 'UF', 'NM_TIP_ACAO',
       'HORA_TRS', 'DIA_MES', 'DIA_PICO', 'DIA_UTIL', 'CONTEXTO', 'APP',
       'HORA_LOCAL', 'DT_ACAO_DATETIME', 'HORA_COMERCIAL',
       'TEMPO_RESPOSTA_SEGUNDOS', 'TIMEOUT'],
      dtype='object')

In [8]:
df.head().T

""
CD_VIS
DT_ACAO
HR_LCL_VIST
TX_IP_VIS
CD_UOR
NM_UOR_RDZ
HR_ACAO
TEMPO_RESPOSTA
TX_IDFC_USU
UF


In [9]:
df.dtypes

CD_VIS                     object
DT_ACAO                    object
HR_LCL_VIST                object
TX_IP_VIS                  object
CD_UOR                     object
NM_UOR_RDZ                 object
HR_ACAO                    object
TEMPO_RESPOSTA             object
TX_IDFC_USU                object
UF                         object
NM_TIP_ACAO                object
HORA_TRS                   object
DIA_MES                    object
DIA_PICO                   object
DIA_UTIL                   object
CONTEXTO                   object
APP                        object
HORA_LOCAL                 object
DT_ACAO_DATETIME           object
HORA_COMERCIAL             object
TEMPO_RESPOSTA_SEGUNDOS    object
TIMEOUT                    object
dtype: object

In [10]:
df.isnull().sum()

CD_VIS                     0
DT_ACAO                    0
HR_LCL_VIST                0
TX_IP_VIS                  0
CD_UOR                     0
NM_UOR_RDZ                 0
HR_ACAO                    0
TEMPO_RESPOSTA             0
TX_IDFC_USU                0
UF                         0
NM_TIP_ACAO                0
HORA_TRS                   0
DIA_MES                    0
DIA_PICO                   0
DIA_UTIL                   0
CONTEXTO                   0
APP                        0
HORA_LOCAL                 0
DT_ACAO_DATETIME           0
HORA_COMERCIAL             0
TEMPO_RESPOSTA_SEGUNDOS    0
TIMEOUT                    0
dtype: int64

In [11]:
df.dropna(subset=['UF', 'NM_TIP_ACAO'],inplace=True)

In [12]:
cores = mp.cpu_count()  #Number of CPU cores on your system
partitions = cores 
print(partitions)

24


In [13]:
def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = mp.Pool(cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [14]:
def getContexto(x):
    if type(x) == float:
        return 'Sem Contexto'
    
    pos = x.find('estatico')
    
    if  pos == -1:
        posInicial = 33
        if x[32:33] == ":":
           posInicial = x.find('/')
        
    else:
        posInicial = pos + 9
        
    ind = x[posInicial:len(x)]
    
    posFinal = ind.find('/')
    contexto = ind[0:posFinal]
    
    if contexto == "":
        contexto = 'Sem Contexto'
    
    return contexto
    

In [15]:
def getApp(x):
    
    if type(x) == float:
        return 'Sem App'
        
    pos = x.find('#')
        
    if pos != -1:
        posFinal = pos
        
    pos = x.find('?')

    if pos != -1:
        posFinal = pos
    else:
        posFinal = len(x)
   
  
    posInicial = getPosInicioApp(x[0:posFinal])    
    
    app = x[posInicial:len(x)]
    
    app = getRetiraSujeira(app)
    
    return app
  

In [16]:
def getRetiraSujeira(x):
    
    posicao = x.find("?")
    app = x
    
    if posicao != -1:
        app = x[0:posicao]
    
    return app    

In [17]:
def getPosInicioApp(x):
    pos = 0
    for i in range(len(x),0,-1):
        ch = x[i-1]
        
        if ch == '/':
            pos = i
            break
            
    return pos

In [18]:
def getHoraLocal(x):
    hora = int(x.HORA_TRS) - 3 
    if hora == -1:
        hora = 23
    elif hora == -2:
        hora = 22
    elif hora == -3:
        hora = 21 
    
    if x.UF in ['AM','RR','RO']:
        hora = int(x.HORA_TRS) - 4
        
        if hora == -1:
            hora = 23
        elif hora == -2:
            hora = 22
        elif hora == -3:
            hora = 21 
        elif hora == -4:
            hora = 20
            
    if x.UF in 'AC':
        hora = int(x.HORA_TRS) - 5
        
        if hora == -1:
            hora = 23
        elif hora == -2:
            hora = 22
        elif hora == -3:
            hora = 21 
        elif hora == -4:
            hora = 20
        elif hora == -5:
            hora = 19
          
    return hora
    

In [19]:
def setDiaUtil(x):
    x['DIA_UTIL'] = 1
    x.loc[x['DT_ACAO'] == '2020-06-06','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-07','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-11','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-13','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-14','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-20','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-21','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-27','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-28','DIA_UTIL'] = 0
    return x

In [20]:
def getDiaUtil(x):
    return datetime.datetime.x.weekday()


In [21]:
def processaArquivo(data): 
    data['HORA_TRS'] = data['HR_ACAO'].str[0:2]  
    data['DIA_MES'] = data['DT_ACAO'].str[8:10]
    
    data['DIA_PICO'] = data.DIA_MES.apply(lambda x: 1 if x in ['30','31','01','02','03','04','05','06','07','08','09','10'] else 0)
   
    data['DIA_UTIL'] = 1
    data.loc[data['DT_ACAO'] == '2020-06-06','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-07','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-11','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-13','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-14','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-20','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-21','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-27','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-28','DIA_UTIL'] = 0
    
    data['NM_TIP_ACAO'] = data.NM_TIP_ACAO.apply(lambda x: x.strip())
    
    data['CONTEXTO'] = data.NM_TIP_ACAO.apply(lambda x: getContexto(x))
                                                 
    data['APP'] = data.NM_TIP_ACAO.apply(lambda x: getApp(x))   
    
    data['HORA_LOCAL'] = data.apply(lambda x: getHoraLocal(x),axis=1)
    data['DT_ACAO_DATETIME'] = pd.to_datetime(data['DT_ACAO'])
    
    data['HORA_COMERCIAL'] = 0
    data.loc[(data['HORA_TRS'] >= '08') | (data['HORA_TRS'] >= '18'),'HORA_COMERCIAL'] = 1
    
    data['TEMPO_RESPOSTA_SEGUNDOS'] = data['TEMPO_RESPOSTA']/1000

    data['TIMEOUT'] = 0
    data.loc[data['TEMPO_RESPOSTA_SEGUNDOS'] > 30,'TIMEOUT'] = 1      
    
     
    return data

In [22]:
df = parallelize(df,processaArquivo)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [23]:
media_temp = df.groupby('APP')['TEMPO_RESPOSTA_SEGUNDOS'].agg({'TEMPO_RESPOSTA_SEGUNDOS':'mean'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS":"Média"})

DataError: No numeric types to aggregate

In [24]:
media_temp.head(20).T

NameError: name 'media_temp' is not defined

In [25]:
media_temp.to_csv('../data/media_apps.csv',sep=';',index=False)

NameError: name 'media_temp' is not defined

In [26]:
teste = pd.qcut(df['TEMPO_RESPOSTA_SEGUNDOS'], q=8, duplicates='drop', precision=2)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [27]:
print(teste)

NameError: name 'teste' is not defined

In [28]:
df['CLASSE_APP'] = pd.cut(df['TEMPO_RESPOSTA_SEGUNDOS'], bins=[-0.01,0.1,0.39, 0.59, 0.92, 1.42, 2.09, 3.3, 30.0,937274])

In [29]:
df.CLASSE_APP.cat.rename_categories({pd.Interval(-0.01, 0.1) : 'Super Baixo (até 0.1 seg)',
                                                      pd.Interval(0.1, 0.39) : 'Muito Baixo (0.1 - 0.39 seg)',
                                                      pd.Interval(0.39, 0.59) : 'Baixo (0.39 - 0.59 seg)',
                                                      pd.Interval(0.59,0.92) : 'Médio (0.59 - 0.92 seg)',
                                                      pd.Interval(0.92,1.42) : 'Médio (0.91 - 1.42 seg)',
                                                      pd.Interval(1.42, 2.09) : 'Alto (1.42 - 2.09 seg)', 
                                                      pd.Interval(2.09, 3.3) : 'Muito Alto (2.09 - 3.3 seg)',
                                                      pd.Interval(3.3, 30) : 'Super Alto (3.24 - 30 seg)',
                                                      pd.Interval(30.0,937274): 'TimeOut'}, 
                                                      inplace=True)

In [30]:
df.head().T

""
CD_VIS
DT_ACAO
HR_LCL_VIST
TX_IP_VIS
CD_UOR
NM_UOR_RDZ
HR_ACAO
TEMPO_RESPOSTA
TX_IDFC_USU
UF


In [31]:
df.isnull().sum()

CD_VIS                     0
DT_ACAO                    0
HR_LCL_VIST                0
TX_IP_VIS                  0
CD_UOR                     0
NM_UOR_RDZ                 0
HR_ACAO                    0
TEMPO_RESPOSTA             0
TX_IDFC_USU                0
UF                         0
NM_TIP_ACAO                0
HORA_TRS                   0
DIA_MES                    0
DIA_PICO                   0
DIA_UTIL                   0
CONTEXTO                   0
APP                        0
HORA_LOCAL                 0
DT_ACAO_DATETIME           0
HORA_COMERCIAL             0
TEMPO_RESPOSTA_SEGUNDOS    0
TIMEOUT                    0
CLASSE_APP                 0
dtype: int64

In [32]:
#temp = df.groupby(['CONTEXTO','APP','NM_TIP_ACAO']).agg({'TEMPO_RESPOSTA_SEGUNDOS':'count'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS": "Qtde"})
#temp = df.groupby(['CONTEXTO','APP','NM_TIP_ACAO']).agg({'TEMPO_RESPOSTA_SEGUNDOS':'count'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS": "Qtde"})
#temp.to_csv('../data/contextos_app.csv',index=False,sep=';')
#temp

In [33]:
df.TEMPO_RESPOSTA_SEGUNDOS.describe()

count       0
unique      0
top       NaN
freq      NaN
Name: TEMPO_RESPOSTA_SEGUNDOS, dtype: object

In [34]:
df.isnull().sum()

CD_VIS                     0
DT_ACAO                    0
HR_LCL_VIST                0
TX_IP_VIS                  0
CD_UOR                     0
NM_UOR_RDZ                 0
HR_ACAO                    0
TEMPO_RESPOSTA             0
TX_IDFC_USU                0
UF                         0
NM_TIP_ACAO                0
HORA_TRS                   0
DIA_MES                    0
DIA_PICO                   0
DIA_UTIL                   0
CONTEXTO                   0
APP                        0
HORA_LOCAL                 0
DT_ACAO_DATETIME           0
HORA_COMERCIAL             0
TEMPO_RESPOSTA_SEGUNDOS    0
TIMEOUT                    0
CLASSE_APP                 0
dtype: int64

In [35]:
df.drop(columns=["NM_TIP_ACAO"],inplace=True)

In [36]:
df.to_csv('../data/base_plataforma_completo_final.csv',sep=';',index=False)